In [1]:
import pandas as pd
import ftfy

In [2]:
# Lendo o dataframe dos emprestimos

df_emprestimos = pd.read_csv('../data/interim/emprestimos-concat.csv')

# Mostrando alguns emprestimos
df_emprestimos.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
0,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2.008023e+09,ALUNO DE GRADUAÇÃO
1,709685,L167050,NaN,2010/01/04 07:44:10.750000000,2010/01/12 07:34:13.934000000,2.008023e+09,ALUNO DE GRADUAÇÃO
2,709686,2006017618,2010/01/26 08:07:01.738000000,2010/01/04 08:08:44.081000000,2010/02/25 07:36:25.800000000,2.008112e+09,ALUNO DE PÓS-GRADUAÇÃO
3,709687,L184117,2010/01/18 11:07:46.470000000,2010/01/04 08:24:21.284000000,2010/02/03 08:58:45.692000000,2.007211e+08,ALUNO DE GRADUAÇÃO
4,709688,2007027500,2010/01/19 15:59:26.464000000,2010/01/04 08:27:33.569000000,2010/02/03 13:06:30.662000000,2.009046e+09,ALUNO DE GRADUAÇÃO


In [ ]:
df_emprestimos.columns

In [3]:
# Lendo o dataframe com os emprestimos
df_exemplares = pd.read_parquet('../data/raw/dados_exemplares.parquet')

# Mostrando alguns exemplares
df_exemplares.head()

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
0,5,L000003,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,694,1
1,4,L000002,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,688,1
2,3,L000001,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,638,1
3,7,L000114,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,616,5
5,10,L000041,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,657,15


In [4]:
# Renomeando as colunas codigo_barras para referenciar aos respectivos dataframes

df_emprestimos.rename(columns={'codigo_barras':'codigo_barras_emprestimo'}, inplace=True)

df_exemplares.rename(columns={'codigo_barras':'codigo_barras_exemplar'}, inplace=True)

In [5]:
df_emprestimos.dtypes

id_emprestimo                 int64
codigo_barras_emprestimo     object
data_renovacao               object
data_emprestimo              object
data_devolucao               object
matricula_ou_siape          float64
tipo_vinculo_usuario         object
dtype: object

In [ ]:
df_exemplares.columns

In [6]:
# Unindo os dataframes

df = pd.merge(df_emprestimos, df_exemplares, left_on='codigo_barras_emprestimo', right_on='codigo_barras_exemplar')

In [ ]:
# Mostrando alguns registros do dataframe unido
df.head()

In [7]:
# Retirando colunas inúteis (registro_sistema)
df.drop(columns=['registro_sistema'], inplace=True)

In [ ]:
df.columns

Os itens do acervo em uma biblioteca são organizados por um sistema de classificação de acordo com o respectivo tema. Existem diversos sistemas, mas este conjunto está de acordo com a CDU - Classificação Decimal Universal. Esta classificação é decimal, pois varia de acordo com a classe de cada assunto:


000 a 099: Generalidades. Ciência e conhecimento.

100 a 199: Filosofia e psicologia.

200 a 299: Religião.

300 a 399: Ciências sociais.

400 a 499: Classe vaga. Provisoriamente não ocupada.

500 a 599: Matemática e ciências naturais.

600 a 699: Ciências aplicadas.

700 a 799: Belas artes.

800 a 899: Linguagem. Língua. Linguística.

900 a 999: Geografia. Biografia. História."

In [8]:
# Classificando o arcevo da biblioteca com .cut()
# o tamanho de rotulos deve ser igual a len(caixas)-1

caixas = [000, 99, 199, 299, 399, 499, 599, 699, 799, 899, 999]

rotulos = [
    'Generalidades. Ciência e conhecimento',
    'Filosofia e psicologia',
    'Religião',
    'Ciências sociais',
    'Classe vaga. Provisoriamente não ocupada',
    'Matemática e ciências naturais',
    'Ciências aplicadas',
    'Belas artes',
    'Linguagem. Língua. Linguística',
    'Geografia. Biografia. História.',
]

df['CDU'] = pd.cut(df['localizacao'], bins=caixas, labels=rotulos, right=True, include_lowest=True)

In [9]:
# A coluna da matricula (“matricula_ou_siape”) não está com um formato muito legível. Transforme-a em formato String.
# O desafio pediu para transformar a coluna em string, mas o pandas assimila uma string como object

df['matricula_ou_siape'] = df['matricula_ou_siape'].astype('string').str.split('.').str[0]

df.dtypes

id_emprestimo                  int64
codigo_barras_emprestimo      object
data_renovacao                object
data_emprestimo               object
data_devolucao                object
matricula_ou_siape            object
tipo_vinculo_usuario          object
id_exemplar                    int64
codigo_barras_exemplar        object
colecao                       object
biblioteca                    object
status_material               object
localizacao                    int64
CDU                         category
dtype: object

In [75]:
df.head()

,id_emprestimo,codigo_barras_emprestimo,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,codigo_barras_exemplar,colecao,biblioteca,status_material,localizacao,CDU
0,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2008023265,ALUNO DE GRADUAÇÃO,13259,L095049,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630,Ciências aplicadas
1,709685,L167050,NaN,2010/01/04 07:44:10.750000000,2010/01/12 07:34:13.934000000,2008023265,ALUNO DE GRADUAÇÃO,70865,L167050,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647,Ciências aplicadas
2,709686,2006017618,2010/01/26 08:07:01.738000000,2010/01/04 08:08:44.081000000,2010/02/25 07:36:25.800000000,2008112114,ALUNO DE PÓS-GRADUAÇÃO,195347,2006017618,Acervo Circulante,Biblioteca Setorial Prof. Rodolfo Helinski - E...,REGULAR,640,Ciências aplicadas
3,709687,L184117,2010/01/18 11:07:46.470000000,2010/01/04 08:24:21.284000000,2010/02/03 08:58:45.692000000,200721119,ALUNO DE GRADUAÇÃO,131639,L184117,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,609,Ciências aplicadas
4,709688,2007027500,2010/01/19 15:59:26.464000000,2010/01/04 08:27:33.569000000,2010/02/03 13:06:30.662000000,2009045971,ALUNO DE GRADUAÇÃO,225646,2007027500,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,686,Ciências aplicadas


In [52]:
df.to_parquet('../data/processed/emprestimos_exemplares.parquet', index=False)

In [70]:
df.to_csv('../data/processed/emprestimos_exemplares.csv', encoding='utf-8-sig', index=False)

In [17]:
df_records_excel_antes_2010 = pd.read_excel(
    '../data/raw/students_records/matricula_alunos.xlsx', sheet_name='Até 2010',skiprows=1
    )

df_records_excel_antes_2010.columns = ['matricula_ou_siape', 'tipo_vinculo_usuario', 'curso']

df_records_excel_antes_2010.columns

Index(['matricula_ou_siape', 'tipo_vinculo_usuario', 'curso'], dtype='object')

In [ ]:
df_records_excel_antes_2010.shape

In [18]:
df_records_excel_apos_2010 = pd.read_excel(
    '../data/raw/students_records/matricula_alunos.xlsx', sheet_name='Após 2010',skiprows=1
    )

df_records_excel_apos_2010.columns = ['matricula_ou_siape', 'tipo_vinculo_usuario', 'curso']

df_records_excel_apos_2010.columns

Index(['matricula_ou_siape', 'tipo_vinculo_usuario', 'curso'], dtype='object')

In [ ]:
df_records_excel_apos_2010.shape

In [19]:
df_records_excel = pd.concat(
    [df_records_excel_antes_2010, df_records_excel_apos_2010],
    ignore_index=True
)

# df_records_excel = df_records_excel.assign(matricula_ou_siape=df_records_excel['matricula_ou_siape'].astype('object'))

df_records_excel.dtypes

matricula_ou_siape      float64
tipo_vinculo_usuario     object
curso                    object
dtype: object

In [20]:
df_records_excel['matricula_ou_siape'] = df_records_excel['matricula_ou_siape'].astype('string').str.split('.').str[0]

In [15]:
df_records_excel.head()

,matricula_ou_siape,tipo_vinculo_usuario,curso
0,190001688,ALUNO DE GRADUAÇÃO,MATEMÁTICA
1,199014489,ALUNO DE GRADUAÇÃO,CIÊNCIAS BIOLÓGICAS
2,199735674,ALUNO DE GRADUAÇÃO,MATEMÁTICA
3,199812431,ALUNO DE GRADUAÇÃO,LETRAS
4,199823670,ALUNO DE GRADUAÇÃO,ECOLOGIA


In [ ]:
df_records_excel.shape

In [ ]:
df_records_excel.to_csv(
    '../data/processed/students_records/matriculas_alunos.csv',
    encoding='utf-8-sig',
    index=False)

In [3]:
df_records_excel_ = pd.read_csv('../data/processed/students_records/matriculas_alunos.csv')

df_records_excel_.columns

Index(['matricula_ou_siape', 'tipo_vinculo_usuario', 'curso'], dtype='object')

In [21]:
# records do ano de 2018

df_records_2018 = pd.read_csv('../data/raw/students_records/discentes-2018.csv')

# df_records_2018.columns

# matricula, nome_curso, nivel_ensino
df_records_2018.dtypes

matricula                 int64
nome_discente            object
sexo                     object
ano_ingresso              int64
periodo_ingresso        float64
forma_ingresso           object
tipo_discente            object
status                   object
sigla_nivel_ensino       object
nivel_ensino             object
id_curso                float64
nome_curso               object
modalidade_educacao      object
id_unidade              float64
nome_unidade             object
id_unidade_gestora      float64
nome_unidade_gestora     object
dtype: object

In [ ]:
df_records_2018.head()

In [ ]:
df_records_2018.isnull().sum()

In [22]:
df_records_2018['nivel_ensino'] = df_records_2018['nivel_ensino'].apply(ftfy.fix_text)
df_records_2018['nivel_ensino'].unique()

array(['GRADUAÇÃO', 'TÉCNICO', 'LATO SENSU', 'MESTRADO',
       'FORMAÇÃO COMPLEMENTAR', 'TÉCNICO INTEGRADO', 'RESIDÊNCIA',
       'DOUTORADO', 'FUNDAMENTAL'], dtype=object)

In [ ]:
df_records_2018['nome_curso'] = df_records_2018['nome_curso'].apply(
    lambda x: ftfy.fix_text(x) if pd.notnull(x) else x
    )
df_records_2018['nome_curso'].unique()

In [ ]:
df_records_2018.head()

In [24]:
df_records_2018_graduacao = df_records_2018.loc[df_records_2018['nivel_ensino'] == 'GRADUAÇÃO', ['matricula','nome_curso','nivel_ensino']]
df_records_2018_graduacao.head()

,matricula,nome_curso,nivel_ensino
0,20180057132,ESTATÍSTICA,GRADUAÇÃO
3,20180107403,CIÊNCIAS E TECNOLOGIA,GRADUAÇÃO
7,20180128824,EDUCAÇÃO FÍSICA,GRADUAÇÃO
10,20180149163,ENGENHARIA CIVIL,GRADUAÇÃO
11,20180073539,HISTÓRIA,GRADUAÇÃO


In [25]:
df_records_2018_graduacao['matricula'] = df_records_2018_graduacao['matricula'].astype('string').str.split('.').str[0]


In [26]:
df_records_2018_graduacao.dtypes

matricula       object
nome_curso      object
nivel_ensino    object
dtype: object

In [28]:
# Records do ano de 2019
df_records_2019 = pd.read_csv('../data/raw/students_records/discentes-2019.csv')
df_records_2019.head()

,matricula,nome_discente,sexo,ano_ingresso,periodo_ingresso,forma_ingresso,tipo_discente,status,sigla_nivel_ensino,nivel_ensino,id_curso,nome_curso,modalidade_educacao,id_unidade,nome_unidade,id_unidade_gestora,nome_unidade_gestora
0,20193012209,ABDENOR BEZERRA DOS SANTOS,M,2019,1.0,PROCESSO SELETIVO,REGULAR,CANCELADO,T,TÃ‰CNICO,96054058.0,CURSO TÃ‰CNICO DA METRÃ“POLE DIGITAL,SEMI-PRESENCIAL,6069.0,INSTITUTO METROPOLE DIGITAL,605.0,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE
1,20190088900,ABDIAS MONTEIRO DE ANDRADE MELO,M,2019,1.0,SiSU,REGULAR,CONCLUÃDO,G,GRADUAÃ‡ÃƒO,111635060.0,HISTÃ“RIA,PRESENCIAL,1482.0,CENTRO DE ENSINO SUPERIOR DO SERIDÃ“,1482.0,CENTRO DE ENSINO SUPERIOR DO SERIDÃ“
2,20190127606,ABDIAS SABINO RODRIGUES FILHO,M,2019,1.0,SiSU,REGULAR,CANCELADO,G,GRADUAÃ‡ÃƒO,111635060.0,HISTÃ“RIA,PRESENCIAL,1482.0,CENTRO DE ENSINO SUPERIOR DO SERIDÃ“,1482.0,CENTRO DE ENSINO SUPERIOR DO SERIDÃ“
3,20195001008,ABEL GOMES DE OLIVEIRA FILHO,M,2019,1.0,PROCESSO SELETIVO,REGULAR,CONCLUÃDO,F,FORMAÃ‡ÃƒO COMPLEMENTAR,112718836.0,CURSO DE LÃNGUA BRASILEIRA DE SINAIS (LIBRAS),PRESENCIAL,6042.0,"INSTITUTO ÃGORA - LÃNGUAS, LITERATURAS E CUL...",442.0,"CENTRO DE CIÃŠNCIAS HUMANAS, LETRAS E ARTES"
4,20190032217,ABI AMANA DE AQUINO BEZERRA,F,2019,2.0,SiSU,REGULAR,CANCELADO,G,GRADUAÃ‡ÃƒO,119512361.0,COMUNICAÃ‡ÃƒO SOCIAL - AUDIOVISUAL,PRESENCIAL,442.0,"CENTRO DE CIÃŠNCIAS HUMANAS, LETRAS E ARTES",442.0,"CENTRO DE CIÃŠNCIAS HUMANAS, LETRAS E ARTES"


In [29]:
df_records_2019['nivel_ensino'] = df_records_2019['nivel_ensino'].apply(ftfy.fix_text)
df_records_2019['nivel_ensino'].unique()

array(['TÉCNICO', 'GRADUAÇÃO', 'FORMAÇÃO COMPLEMENTAR', 'DOUTORADO',
       'LATO SENSU', 'MESTRADO', 'RESIDÊNCIA', 'TÉCNICO INTEGRADO',
       'FUNDAMENTAL'], dtype=object)

In [ ]:
df_records_2019['nome_curso'] = df_records_2019['nome_curso'].apply(
    lambda x: ftfy.fix_text(x) if pd.notnull(x) else x
    )
df_records_2019['nome_curso'].unique()

In [31]:
df_records_2019_graduacao = df_records_2019.loc[df_records_2019['nivel_ensino'] == 'GRADUAÇÃO', ['matricula','nome_curso','nivel_ensino']]
df_records_2019_graduacao.head()

,matricula,nome_curso,nivel_ensino
1,20190088900,HISTÓRIA,GRADUAÇÃO
2,20190127606,HISTÓRIA,GRADUAÇÃO
4,20190032217,COMUNICAÇÃO SOCIAL - AUDIOVISUAL,GRADUAÇÃO
6,20190015083,SAÚDE COLETIVA,GRADUAÇÃO
7,20190013409,FARMÁCIA,GRADUAÇÃO


In [34]:
df_records_2019_graduacao['matricula'] = df_records_2019_graduacao['matricula'].astype('string').str.split('.').str[0]

In [35]:
df_records_2019_graduacao.dtypes

matricula       object
nome_curso      object
nivel_ensino    object
dtype: object

In [36]:
df_records_2018_2019 = pd.concat(
    [df_records_2018_graduacao, df_records_2019_graduacao],
    ignore_index=True
)

In [37]:
df_records_2018_2019.dtypes

matricula       object
nome_curso      object
nivel_ensino    object
dtype: object

In [38]:
df_records_2018_2019.columns = ['matricula_ou_siape', 'curso', 'tipo_vinculo_usuario']

In [39]:
df_records_2018_2019.head()

,matricula_ou_siape,curso,tipo_vinculo_usuario
0,20180057132,ESTATÍSTICA,GRADUAÇÃO
1,20180107403,CIÊNCIAS E TECNOLOGIA,GRADUAÇÃO
2,20180128824,EDUCAÇÃO FÍSICA,GRADUAÇÃO
3,20180149163,ENGENHARIA CIVIL,GRADUAÇÃO
4,20180073539,HISTÓRIA,GRADUAÇÃO


In [40]:
vinculo = {
    'GRADUAÇÃO': 'ALUNO DE GRADUAÇÃO'
}

df_records_2018_2019['tipo_vinculo_usuario'] = df_records_2018_2019['tipo_vinculo_usuario'].map(vinculo)

df_records_2018_2019.head()

,matricula_ou_siape,curso,tipo_vinculo_usuario
0,20180057132,ESTATÍSTICA,ALUNO DE GRADUAÇÃO
1,20180107403,CIÊNCIAS E TECNOLOGIA,ALUNO DE GRADUAÇÃO
2,20180128824,EDUCAÇÃO FÍSICA,ALUNO DE GRADUAÇÃO
3,20180149163,ENGENHARIA CIVIL,ALUNO DE GRADUAÇÃO
4,20180073539,HISTÓRIA,ALUNO DE GRADUAÇÃO


In [21]:
df_records_2018_2019.to_csv(
    '../data/processed/students_records/matricula_alunos_2018_2019.csv',
    encoding='utf-8-sig',
    index=False
)

In [42]:
df_matriculas = pd.concat(
    [df_records_excel, df_records_2018_2019],
    ignore_index=True
)

In [43]:
df_matriculas.head()

,matricula_ou_siape,tipo_vinculo_usuario,curso
0,190001688,ALUNO DE GRADUAÇÃO,MATEMÁTICA
1,199014489,ALUNO DE GRADUAÇÃO,CIÊNCIAS BIOLÓGICAS
2,199735674,ALUNO DE GRADUAÇÃO,MATEMÁTICA
3,199812431,ALUNO DE GRADUAÇÃO,LETRAS
4,199823670,ALUNO DE GRADUAÇÃO,ECOLOGIA


In [24]:
df_matriculas.dtypes

matricula_ou_siape      object
tipo_vinculo_usuario    object
curso                   object
dtype: object

In [ ]:
df_matriculas.shape

In [25]:
df_matriculas.to_csv(
    '../data/processed/students_records/matriculas_alunos_total.csv',
    encoding='utf-8-sig',
    index=False
)

In [44]:
df_records_json = pd.read_json('../data/raw/students_records/cadastro_alunos.json')

df_records_json_registros = pd.read_json(df_records_json.registros[0])

df_records_json_registros = df_records_json_registros.assign(matricula_ou_siape=df_records_json_registros['matricula_ou_siape'].astype('object'))

df_records_json_registros.dtypes

F:\pedro\Documents\dev\VSCode\VSCode Python\data\tmp\ipykernel_10852\2815048638.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_records_json_registros = pd.read_json(df_records_json.registros[0])


matricula_ou_siape      object
tipo_vinculo_usuario    object
curso                   object
dtype: object

In [ ]:
df_records_json_registros.shape

In [ ]:
df_records_json_registros.head()

In [45]:
df_records = pd.concat([
    df_matriculas, df_records_json_registros
],ignore_index=True)

In [ ]:
df_records.shape

In [46]:
df_records.dtypes

matricula_ou_siape      object
tipo_vinculo_usuario    object
curso                   object
dtype: object

In [48]:
df_records['matricula_ou_siape'] = df_records['matricula_ou_siape'].astype('string').str.split('.').str[0]

In [46]:
df_records.head()

,matricula_ou_siape,tipo_vinculo_usuario,curso
0,190001688,ALUNO DE GRADUAÇÃO,MATEMÁTICA
1,199014489,ALUNO DE GRADUAÇÃO,CIÊNCIAS BIOLÓGICAS
2,199735674,ALUNO DE GRADUAÇÃO,MATEMÁTICA
3,199812431,ALUNO DE GRADUAÇÃO,LETRAS
4,199823670,ALUNO DE GRADUAÇÃO,ECOLOGIA


In [28]:
df_records.to_csv('../data/processed/students_records/records.csv', index=False, encoding='utf-8-sig')

In [51]:
df_records.to_parquet('../data/processed/students_records/records.parquet', index=False)

In [ ]:
records = pd.read_csv('../data/processed/students_records/records.csv')

records.dtypes